### Warm-ups

1. **Make profile with defaults**

In [1]:
def make_profile(name, email=None, country="IN"):
    """Return user profile dict with given name, email (optional), country (default "IN")."""
    profile = {"name": name, "country": country}
    if email is not None:
        profile["email"] = email
    return profile

p = make_profile("A","a@example.com")
assert p["name"]=="A" and p["country"]=="IN"

2. **Safe get path** — nested dict path or default

In [2]:
def get_in(d, path, default=None):
    """Return d[path[0]][path[1]]... or default if any key is missing."""
    for key in path:
        if not isinstance(d, dict) or key not in d:
            return default
        d = d[key]
    return d

cfg = {"db":{"host":"localhost","port":5432}}
assert get_in(cfg, ["db","port"]) == 5432
assert get_in(cfg, ["db","user"], "postgres") == "postgres"

3. **Set with default container**

In [3]:
def multimap_add(mm, key, value):
    """mm: dict key -> list; append value; create list if missing."""
    if key not in mm:
        mm[key] = []
    mm[key].append(value)
    
mm = {}
multimap_add(mm,"a",1); multimap_add(mm,"a",2)
assert mm == {"a":[1,2]}

### Core

4. **Invert mapping (values unique)**
   If duplicate values appear, keep first key.

In [4]:
def invert_unique(d):
    """Return dict with keys and values swapped; assume values are unique."""
    return {v: k for k, v in d.items()}

assert invert_unique({"a":1,"b":2}) == {1:"a",2:"b"}

5. **Invert to sets (values may repeat)**

In [5]:
def invert_to_sets(d):
    """Return dict with keys and values swapped; values become sets of keys."""
    inv = {}
    for k, v in d.items():
        if v not in inv:
            inv[v] = set()
        inv[v].add(k)
    return inv

assert invert_to_sets({"a":1,"b":1,"c":2}) == {1:{"a","b"}, 2:{"c"}}

6. **Merge with priority** — `override` wins

In [6]:
def merge_priority(base, override):
    """Return new dict with keys/values from base updated by override."""
    merged = base.copy()
    merged.update(override)
    return merged

assert merge_priority({"a":1,"b":2},{"b":9,"c":3}) == {"a":1,"b":9,"c":3}

7. **Group by key function**

In [7]:
def group_by(xs, key_fn):
    """Return dict key -> list of items in xs with that key (from key_fn)."""
    grouped = {}
    for x in xs:
        key = key_fn(x)
        if key not in grouped:
            grouped[key] = []
        grouped[key].append(x)
    return grouped

g = group_by(["apple","pear","plum","kiwi"], len)
assert g == {5:["apple"],4:["pear","plum","kiwi"]}

8. **Top-N by value**

In [8]:
def top_n(d, n):
    """Return list of (key,value) pairs for top n largest values in d."""
    return sorted(d.items(), key=lambda item: item[1], reverse=True)[:n]

assert top_n({"a":3,"b":1,"c":5},2) == [("c",5),("a",3)]

9. **Join two “tables” by key** (left join)

In [9]:
def left_join(left_rows, right_rows, key):
    """
    left_rows/right_rows: list of dicts
    Return: list of merged dicts; right fields may be missing.
    """
    right_index = {row[key]: row for row in right_rows}
    result = []
    for lrow in left_rows:
        rrow = right_index.get(lrow[key], {})
        merged = lrow.copy()
        merged.update(rrow)
        result.append(merged)
    return result

L = [{"id":1,"name":"A"},{"id":2,"name":"B"}]
R = [{"id":1,"score":90}]
out = left_join(L,R,"id")
assert out[0]["score"]==90 and out[1].get("score") is None

### Challenge

10. **Flatten nested dict with dot paths**

In [10]:
def flatten_dot(d, parent_key=""):
    """
    {"a":{"b":1},"c":2} -> {"a.b":1, "c":2}
    """
    items = {}
    for k, v in d.items():
        new_key = f"{parent_key}.{k}" if parent_key else k
        if isinstance(v, dict):
            items.update(flatten_dot(v, new_key))
        else:
            items[new_key] = v
    return items

assert flatten_dot({"a":{"b":1},"c":2}) == {"a.b":1,"c":2}